In [102]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

In [103]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [104]:
test_index=test.shape[0]
train_index=train.shape[0]
train_ID = train['Id']
test_ID = test['Id']
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)
train_raw_labels = train['SalePrice'].to_frame().as_matrix()
train_labels=train['SalePrice'].values
concated_dataset=pd.concat([train,test])
concated_dataset.drop(['SalePrice'], axis=1, inplace=True)



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [105]:
test.shape

(1459, 79)

In [106]:
train_raw_labels.shape

(1460, 1)

In [107]:
concated_dataset.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,NaN,3,1Fam,TA,No,706.0,0.0,...,WD,0,Pave,8,856.0,AllPub,0,2003,2003,2008
1,1262,0,0,NaN,3,1Fam,TA,Gd,978.0,0.0,...,WD,0,Pave,6,1262.0,AllPub,298,1976,1976,2007
2,920,866,0,NaN,3,1Fam,TA,Mn,486.0,0.0,...,WD,0,Pave,6,920.0,AllPub,0,2001,2002,2008
3,961,756,0,NaN,3,1Fam,Gd,No,216.0,0.0,...,WD,0,Pave,7,756.0,AllPub,0,1915,1970,2006
4,1145,1053,0,NaN,4,1Fam,TA,Av,655.0,0.0,...,WD,0,Pave,9,1145.0,AllPub,192,2000,2000,2008


In [108]:
all_data_na = (concated_dataset.isnull().sum() / len(concated_dataset)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

,Missing Ratio
PoolQC,99.657417
MiscFeature,96.402878
Alley,93.216855
Fence,80.438506
FireplaceQu,48.646797
LotFrontage,16.649538
GarageQual,5.447071
GarageCond,5.447071
GarageFinish,5.447071
GarageYrBlt,5.447071


From the list we can begin working with data filling.First let check what is PoolQC and why this is empthy.This have four values Ex=Excellent,Gd=Good,TA=typical,Fa=Fair,NA=no pool.So may be missing data should be NA.Lets also help our model by just replacing with neumerical values I will give Ex=4 and No Pool=0. Since the saleprice should be higher if the PoolQC is more.

In [109]:
concated_dataset["PoolQC"] = concated_dataset["PoolQC"].fillna(0)
concated_dataset["PoolQC"] =concated_dataset["PoolQC"].replace(to_replace={"Ex": 4, "Gd":3,"TA":2,"Fa":1})


Lets verify 

In [110]:
for i in concated_dataset["PoolQC"] :
    if i !=0:
        print(i)

4
1
3
4
3
1
3
4
4
3


Now this looks good. Now we can investigate next feature. Which MiscFeature is 96% missing data.This has Elev {Elevator}, Gar2 {2nd Garage} ,Othr {Other},Shed {Shed (over 100 SF}TenC {Tennis Court},NA   {None} values.But we can but keep NA lets replace. We also need to make it catagorical at some point.

In [111]:
concated_dataset["MiscFeature"] = concated_dataset["MiscFeature"].fillna("None")


In [112]:
concated_dataset["MiscFeature"]=pd.Categorical(concated_dataset["MiscFeature"])

In [113]:
dfDummies1 = pd.get_dummies(concated_dataset["MiscFeature"], prefix = 'MiscFeature')
concated_dataset = pd.concat([concated_dataset, dfDummies1], axis=1)
concated_dataset = concated_dataset.drop(['MiscFeature'], axis=1)


In [114]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
lbl = LabelEncoder() 


Now lets fill up Alley.This can be of 3 types Gravel, paved and No Alley which means NA will have different meaning.

In [115]:
concated_dataset["Alley"] = concated_dataset["Alley"].fillna("None")
concated_dataset["Alley"] =concated_dataset["Alley"].replace(to_replace={"Grvl": 2, "Pave":1,"None":0})


In [116]:
for i in concated_dataset["Alley"] :
    if i !=0:
        print(i)

2
1
1
2
1
2
2
1
1
2
2
2
2
1
1
2
1
2
2
1
1
1
1
2
2
2
2
2
2
1
1
2
2
1
1
2
2
1
2
2
1
2
2
1
1
1
2
2
2
1
2
1
1
2
2
1
1
1
1
2
2
2
1
1
2
2
2
1
1
2
1
2
2
1
2
2
1
1
2
2
1
1
2
2
2
1
2
1
2
2
1
1
1
1
1
1
1
2
2
2
2
2
2
2
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
1
1
1
1
1
1
1
2
2
2
1
1
2
2
2
2
1
2
2
2
2
2
2
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
1
2
2
2


In [117]:
concated_dataset["Fence"] = concated_dataset["Fence"].fillna("None")



In [118]:
concated_dataset["Fence"] =pd.Categorical(concated_dataset["Fence"])
dfDummies2 = pd.get_dummies(concated_dataset["Fence"], prefix = 'Fence')
concated_dataset = pd.concat([concated_dataset, dfDummies2], axis=1)
concated_dataset = concated_dataset.drop(['Fence'], axis=1)


Now lets find out how to encode fireplace. From the descreption 
   Ex   Excellent - Exceptional Masonry Fireplace
   Gd   Good - Masonry Fireplace in main level
   TA   Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
   Fa   Fair - Prefabricated Fireplace in basement
   Po   Poor - Ben Franklin Stove
   NA   No Fireplace
So we can replace Ex with 5 and NA with 0.


In [119]:
concated_dataset["FireplaceQu"] = concated_dataset["FireplaceQu"].fillna("None")
concated_dataset["FireplaceQu"] =concated_dataset["FireplaceQu"].replace(to_replace={"Ex": 5, "Gd":4,"TA":3,"Fa":2,"Po":1,"None":0})


In [120]:
for i in concated_dataset["FireplaceQu"] :
    if i !=0:
        print(i)

3
3
4
3
4
3
3
3
4
4
2
3
4
4
4
3
3
4
4
4
4
4
4
3
3
4
4
5
4
4
3
4
4
4
4
4
3
4
3
4
4
3
3
4
4
3
3
3
3
4
3
3
4
4
3
2
1
3
4
3
3
3
3
2
4
3
3
4
2
3
1
4
4
4
4
4
4
4
4
4
4
4
4
3
4
3
3
3
4
3
4
4
3
4
4
3
3
4
3
3
4
5
4
2
4
3
1
4
3
2
3
3
3
5
3
2
3
3
1
3
4
3
2
3
3
3
4
3
4
3
3
4
3
3
3
4
4
2
4
3
4
4
5
3
3
4
4
4
4
4
3
4
3
4
4
4
3
3
5
3
5
3
4
4
4
4
4
3
4
3
3
4
3
4
4
4
4
3
4
4
5
4
4
5
4
3
4
1
3
3
5
3
4
4
4
4
3
4
4
4
5
3
4
4
2
3
2
5
3
3
3
4
4
3
3
3
4
4
4
4
3
3
4
2
3
3
4
4
3
3
3
3
4
4
4
4
3
4
4
3
4
3
3
4
4
3
1
4
3
1
4
4
4
5
3
4
4
4
4
4
4
3
4
4
3
2
4
3
2
3
4
3
4
4
3
3
4
4
4
4
3
3
4
3
2
1
4
2
3
4
3
3
4
4
4
4
4
4
3
3
3
4
3
4
4
3
4
4
3
3
4
2
4
4
4
4
4
4
3
4
3
3
4
4
3
3
2
4
5
3
3
4
4
4
3
3
3
3
4
3
4
3
1
4
4
3
3
4
4
3
4
3
3
4
4
3
3
3
5
3
1
2
3
4
3
3
4
3
4
3
4
3
3
4
4
3
5
4
3
4
4
4
1
3
4
4
4
3
2
3
1
3
4
4
3
4
3
3
3
4
3
3
3
3
4
3
4
3
3
4
4
4
3
4
4
3
4
2
3
3
3
4
4
4
3
3
3
3
3
4
3
4
4
3
4
2
4
4
4
4
4
4
4
4
3
4
3
1
4
3
3
2
4
3
3
4
2
4
3
3
4
4
4
3
3
4
4
4
3
3
3
3
3
1
3
4
3
4
3
4
4
4
4
4
3
3
4
2
3
4
4
3
3
4
4
4
3
3
4
3


LotFrontage: Linear feet of street connected to property.NA probably means not given. LotFrontage : Since the area of each street connected to the house property most likely have a similar area to other houses in its neighborhood , we can fill in missing values by the median LotFrontage of the neighborhood.

In [121]:
concated_dataset["LotFrontage"] = concated_dataset.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))


In [122]:
print(concated_dataset["LotFrontage"])

0        65.0
1        80.0
2        68.0
3        60.0
4        84.0
5        85.0
6        75.0
7        80.0
8        51.0
9        50.0
10       70.0
11       85.0
12       72.0
13       91.0
14       73.0
15       51.0
16       73.0
17       72.0
18       66.0
19       70.0
20      101.0
21       57.0
22       75.0
23       44.0
24       72.0
25      110.0
26       60.0
27       98.0
28       47.0
29       60.0
        ...  
1429     50.0
1430     75.0
1431     69.0
1432     50.0
1433     60.0
1434     41.0
1435     44.0
1436     69.0
1437     65.0
1438     70.0
1439    140.0
1440     82.0
1441     82.0
1442     95.0
1443     88.0
1444    125.0
1445     78.0
1446     41.0
1447     58.0
1448     74.0
1449     21.0
1450     21.0
1451     80.0
1452     21.0
1453     21.0
1454     21.0
1455     21.0
1456    160.0
1457     62.0
1458     74.0
Name: LotFrontage, Length: 2919, dtype: float64


GarageType, GarageFinish, GarageQual, GarageCond is NA if no Garage is there. NA should be replaced with None.

In [123]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    concated_dataset[col] = concated_dataset[col].fillna('None')

Like previous I will fill up manually since this will help the netwok to tune easily. 

In [124]:
concated_dataset["GarageFinish"] =concated_dataset["GarageFinish"].replace(to_replace={"Fin": 3, "RFn":2,"Unf":1,"None":0})


In [125]:
print(concated_dataset["GarageFinish"])

0       2
1       2
2       2
3       1
4       2
5       1
6       2
7       2
8       1
9       2
10      1
11      3
12      1
13      2
14      2
15      1
16      3
17      1
18      1
19      1
20      2
21      1
22      2
23      1
24      1
25      2
26      1
27      2
28      2
29      1
       ..
1429    1
1430    1
1431    0
1432    0
1433    0
1434    3
1435    3
1436    3
1437    1
1438    2
1439    1
1440    1
1441    1
1442    3
1443    3
1444    1
1445    3
1446    2
1447    2
1448    1
1449    0
1450    1
1451    2
1452    1
1453    0
1454    0
1455    1
1456    1
1457    0
1458    3
Name: GarageFinish, Length: 2919, dtype: int64


In [126]:
concated_dataset["GarageQual"] =concated_dataset["GarageQual"].replace(to_replace={"Ex": 5, "Gd":4,"TA":3,"Fa":2,"Po":1,"None":0})


In [127]:
concated_dataset["GarageCond"] =concated_dataset["GarageCond"].replace(to_replace={"Ex": 5, "Gd":4,"TA":3,"Fa":2,"Po":1,"None":0})


In [128]:
print(concated_dataset["GarageQual"] )

0       3
1       3
2       3
3       3
4       3
5       3
6       3
7       3
8       2
9       4
10      3
11      3
12      3
13      3
14      3
15      3
16      3
17      3
18      3
19      3
20      3
21      3
22      3
23      3
24      3
25      3
26      3
27      3
28      3
29      2
       ..
1429    3
1430    3
1431    0
1432    0
1433    0
1434    3
1435    3
1436    3
1437    3
1438    3
1439    3
1440    3
1441    3
1442    3
1443    3
1444    2
1445    3
1446    3
1447    3
1448    3
1449    0
1450    3
1451    3
1452    3
1453    0
1454    0
1455    3
1456    3
1457    0
1458    3
Name: GarageQual, Length: 2919, dtype: int64


In [129]:
concated_dataset["GarageType"]=pd.Categorical(concated_dataset["GarageType"])

In [130]:
dfDummies = pd.get_dummies(concated_dataset["GarageType"], prefix = 'GarageType')


In [131]:
concated_dataset = pd.concat([concated_dataset, dfDummies], axis=1)
concated_dataset = concated_dataset.drop(['GarageType'], axis=1)


In [132]:
concated_dataset.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,Fence_MnPrv,Fence_MnWw,Fence_None,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None
0,856,854,0,0,3,1Fam,TA,No,706.0,0.0,...,0,0,1,0,1,0,0,0,0,0
1,1262,0,0,0,3,1Fam,TA,Gd,978.0,0.0,...,0,0,1,0,1,0,0,0,0,0
2,920,866,0,0,3,1Fam,TA,Mn,486.0,0.0,...,0,0,1,0,1,0,0,0,0,0
3,961,756,0,0,3,1Fam,Gd,No,216.0,0.0,...,0,0,1,0,0,0,0,0,1,0
4,1145,1053,0,0,4,1Fam,TA,Av,655.0,0.0,...,0,0,1,0,1,0,0,0,0,0


In [133]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    concated_dataset[col] = concated_dataset[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    concated_dataset[col] = concated_dataset[col].fillna(0)

In [134]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    concated_dataset[col] = concated_dataset[col].fillna('None')

In [135]:
concated_dataset["BsmtQual"] =concated_dataset["BsmtQual"].replace(to_replace={"Ex": 5, "Gd":4,"TA":3,"Fa":2,"Po":1,"None":0})


In [136]:
concated_dataset["BsmtCond"] =concated_dataset["BsmtCond"].replace(to_replace={"Ex": 5, "Gd":4,"TA":3,"Fa":2,"Po":1,"None":0})


In [137]:
concated_dataset["BsmtExposure"] =concated_dataset["BsmtExposure"].replace(to_replace={"Gd":3,"Av":2,"Mn":1,"No":0,"None":0})


In [138]:
concated_dataset["BsmtFinType1"] =concated_dataset["BsmtFinType1"].replace(to_replace={"GLQ":6,"ALQ":5,"BLQ":4,"Rec":3,"LwQ":2,"Unf":1,"None":0})
concated_dataset["BsmtFinType2"] =concated_dataset["BsmtFinType2"].replace(to_replace={"GLQ":6,"ALQ":5,"BLQ":4,"Rec":3,"LwQ":2,"Unf":1,"None":0})


In [139]:
print(concated_dataset["BsmtFinType1"])

0       6
1       5
2       6
3       5
4       6
5       6
6       6
7       5
8       1
9       6
10      3
11      6
12      5
13      1
14      4
15      1
16      5
17      0
18      6
19      2
20      1
21      1
22      1
23      6
24      3
25      1
26      4
27      6
28      4
29      1
       ..
1429    1
1430    1
1431    0
1432    1
1433    1
1434    6
1435    6
1436    6
1437    1
1438    1
1439    6
1440    4
1441    6
1442    6
1443    6
1444    0
1445    6
1446    1
1447    5
1448    4
1449    6
1450    3
1451    3
1452    3
1453    1
1454    1
1455    3
1456    5
1457    6
1458    2
Name: BsmtFinType1, Length: 2919, dtype: int64


In [140]:
concated_dataset["MasVnrType"] = concated_dataset["MasVnrType"].fillna("None")
concated_dataset["MasVnrArea"] = concated_dataset["MasVnrArea"].fillna(0)

In [141]:
concated_dataset["MasVnrType"]=pd.Categorical(concated_dataset["MasVnrType"])
dfDummies_MasVnrType = pd.get_dummies(concated_dataset["MasVnrType"], prefix = 'MasVnrType')
concated_dataset = pd.concat([concated_dataset, dfDummies_MasVnrType], axis=1)
concated_dataset = concated_dataset.drop(['MasVnrType'], axis=1)


In [142]:
concated_dataset.select_dtypes(include=['object']).columns


Index(['BldgType', 'CentralAir', 'Condition1', 'Condition2', 'Electrical',
       'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd', 'Foundation',
       'Functional', 'Heating', 'HeatingQC', 'HouseStyle', 'KitchenQual',
       'LandContour', 'LandSlope', 'LotConfig', 'LotShape', 'MSZoning',
       'Neighborhood', 'PavedDrive', 'RoofMatl', 'RoofStyle', 'SaleCondition',
       'SaleType', 'Street', 'Utilities'],
      dtype='object')

In [143]:
concated_dataset['MSZoning'] = concated_dataset['MSZoning'].fillna(concated_dataset['MSZoning'].mode()[0])


In [144]:
concated_dataset["MSZoning"]=pd.Categorical(concated_dataset["MSZoning"])
dfDummies_MSZoning = pd.get_dummies(concated_dataset["MSZoning"], prefix = 'MSZoning')
concated_dataset = pd.concat([concated_dataset, dfDummies_MSZoning], axis=1)
concated_dataset = concated_dataset.drop(['MSZoning'], axis=1)


In [145]:
concated_dataset["Utilities"] = concated_dataset["Utilities"].fillna("None")


In [146]:
concated_dataset["Utilities"]=pd.Categorical(concated_dataset["Utilities"])
dfDummies_Utilities = pd.get_dummies(concated_dataset["Utilities"], prefix = 'Utilities')
concated_dataset = pd.concat([concated_dataset, dfDummies_Utilities], axis=1)
concated_dataset = concated_dataset.drop(['Utilities'], axis=1)


In [147]:
concated_dataset["Functional"] = concated_dataset["Functional"].fillna("Typ")


In [148]:
concated_dataset["Functional"] =concated_dataset["Functional"].replace(to_replace={"Typ": 8, "Min1":7,"Min2":6,"Mod":5,"Maj1":4,"Maj2":3,"Sev":2,"Sal":1})


In [149]:
concated_dataset['Electrical'] = concated_dataset['Electrical'].fillna(concated_dataset['Electrical'].mode()[0])


In [150]:
concated_dataset["Electrical"]=pd.Categorical(concated_dataset["Electrical"])
dfDummies_Electrical = pd.get_dummies(concated_dataset["Electrical"], prefix = 'Electrical')
concated_dataset = pd.concat([concated_dataset, dfDummies_Electrical], axis=1)
concated_dataset = concated_dataset.drop(['Electrical'], axis=1)


In [151]:
concated_dataset['KitchenQual'] = concated_dataset['KitchenQual'].fillna(concated_dataset['KitchenQual'].mode()[0])


In [152]:
concated_dataset["KitchenQual"] =concated_dataset["KitchenQual"].replace(to_replace={"Ex": 5, "Gd":4,"TA":3,"Fa":2,"Po":1,"None":0})


In [153]:
concated_dataset['Exterior1st'] = concated_dataset['Exterior1st'].fillna(concated_dataset['Exterior1st'].mode()[0])
concated_dataset['Exterior2nd'] = concated_dataset['Exterior2nd'].fillna(concated_dataset['Exterior2nd'].mode()[0])

In [154]:
concated_dataset["Exterior1st"]=pd.Categorical(concated_dataset["Exterior1st"])
dfDummies_Exterior1st = pd.get_dummies(concated_dataset["Exterior1st"], prefix = 'Exterior1st')
concated_dataset = pd.concat([concated_dataset, dfDummies_Exterior1st], axis=1)
concated_dataset = concated_dataset.drop(['Exterior1st'], axis=1)


In [155]:
concated_dataset["Exterior2nd"]=pd.Categorical(concated_dataset["Exterior2nd"])
dfDummies_Exterior2nd = pd.get_dummies(concated_dataset["Exterior2nd"], prefix = 'Exterior2nd')
concated_dataset = pd.concat([concated_dataset, dfDummies_Exterior2nd], axis=1)
concated_dataset = concated_dataset.drop(['Exterior2nd'], axis=1)


In [156]:
concated_dataset['SaleType'] = concated_dataset['SaleType'].fillna(concated_dataset['SaleType'].mode()[0])


In [157]:
concated_dataset["SaleType"]=pd.Categorical(concated_dataset["SaleType"])
dfDummies_SaleType = pd.get_dummies(concated_dataset["SaleType"], prefix = 'SaleType')
concated_dataset = pd.concat([concated_dataset, dfDummies_SaleType], axis=1)
concated_dataset = concated_dataset.drop(['SaleType'], axis=1)


In [158]:
concated_dataset['MSSubClass'] = concated_dataset['MSSubClass'].astype(str)
concated_dataset['MSSubClass'] = concated_dataset['MSSubClass'].fillna(concated_dataset['MSSubClass'].mode()[0])



In [159]:
concated_dataset["MSSubClass"]=pd.Categorical(concated_dataset["MSSubClass"])
dfDummies_MSSubClass = pd.get_dummies(concated_dataset["MSSubClass"], prefix = 'MSSubClass')
concated_dataset = pd.concat([concated_dataset, dfDummies_MSSubClass], axis=1)
concated_dataset = concated_dataset.drop(['MSSubClass'], axis=1)


In [160]:
all_data_na = (concated_dataset.isnull().sum() / len(concated_dataset)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

,Missing Ratio


In [161]:
concated_dataset.select_dtypes(include=['object']).columns


Index(['BldgType', 'CentralAir', 'Condition1', 'Condition2', 'ExterCond',
       'ExterQual', 'Foundation', 'Heating', 'HeatingQC', 'HouseStyle',
       'LandContour', 'LandSlope', 'LotConfig', 'LotShape', 'Neighborhood',
       'PavedDrive', 'RoofMatl', 'RoofStyle', 'SaleCondition', 'Street'],
      dtype='object')

In [162]:
concated_dataset["BldgType"]=pd.Categorical(concated_dataset["BldgType"])
dfDummies_BldgType = pd.get_dummies(concated_dataset["BldgType"], prefix = 'BldgType')
concated_dataset = pd.concat([concated_dataset, dfDummies_BldgType], axis=1)
concated_dataset = concated_dataset.drop(['BldgType'], axis=1)


In [163]:
concated_dataset["CentralAir"] =concated_dataset["CentralAir"].replace(to_replace={"N": 5, "Y":4})


In [164]:
concated_dataset["Condition1"]=pd.Categorical(concated_dataset["Condition1"])
dfDummies_condition1 = pd.get_dummies(concated_dataset["Condition1"], prefix = 'Condition1')
concated_dataset = pd.concat([concated_dataset, dfDummies_condition1], axis=1)
concated_dataset = concated_dataset.drop(['Condition1'], axis=1)


In [165]:
concated_dataset["Condition2"]=pd.Categorical(concated_dataset["Condition2"])
dfDummies_condition2 = pd.get_dummies(concated_dataset["Condition2"], prefix = 'Condition2')
concated_dataset = pd.concat([concated_dataset, dfDummies_condition2], axis=1)
concated_dataset = concated_dataset.drop(['Condition2'], axis=1)


In [166]:
concated_dataset["ExterCond"] =concated_dataset["ExterCond"].replace(to_replace={"Ex": 5, "Gd":4,"TA":3,"Fa":2,"Po":1,"None":0})


In [167]:
concated_dataset["ExterQual"] =concated_dataset["ExterQual"].replace(to_replace={"Ex": 5, "Gd":4,"TA":3,"Fa":2,"Po":1,"None":0})


In [168]:
concated_dataset["Foundation"]=pd.Categorical(concated_dataset["Foundation"])
dfDummies_Foundation = pd.get_dummies(concated_dataset["Foundation"], prefix = 'Foundation')
concated_dataset = pd.concat([concated_dataset, dfDummies_Foundation], axis=1)
concated_dataset = concated_dataset.drop(['Foundation'], axis=1)


In [169]:
concated_dataset["Heating"]=pd.Categorical(concated_dataset["Heating"])
dfDummies_Heating = pd.get_dummies(concated_dataset["Heating"], prefix = 'Heating')
concated_dataset = pd.concat([concated_dataset, dfDummies_Heating], axis=1)
concated_dataset = concated_dataset.drop(['Heating'], axis=1)


In [170]:
concated_dataset["HeatingQC"] =concated_dataset["HeatingQC"].replace(to_replace={"Ex": 5, "Gd":4,"TA":3,"Fa":2,"Po":1})


In [171]:
concated_dataset["HouseStyle"]=pd.Categorical(concated_dataset["HouseStyle"])
dfDummies_HouseStyle = pd.get_dummies(concated_dataset["HouseStyle"], prefix = 'HouseStyle')
concated_dataset = pd.concat([concated_dataset, dfDummies_HouseStyle], axis=1)
concated_dataset = concated_dataset.drop(['HouseStyle'], axis=1)


In [172]:
concated_dataset["LandContour"]=pd.Categorical(concated_dataset["LandContour"])
dfDummies_LandContour = pd.get_dummies(concated_dataset["LandContour"], prefix = 'LandContour')
concated_dataset = pd.concat([concated_dataset, dfDummies_LandContour], axis=1)
concated_dataset = concated_dataset.drop(['LandContour'], axis=1)


In [173]:
concated_dataset["LandSlope"]=pd.Categorical(concated_dataset["LandSlope"])
dfDummies_LandSlope = pd.get_dummies(concated_dataset["LandSlope"], prefix = 'LandSlope')
concated_dataset = pd.concat([concated_dataset, dfDummies_LandSlope], axis=1)
concated_dataset = concated_dataset.drop(['LandSlope'], axis=1)


In [174]:
concated_dataset["LotConfig"]=pd.Categorical(concated_dataset["LotConfig"])
dfDummies_LotConfig = pd.get_dummies(concated_dataset["LotConfig"], prefix = 'LotConfig')
concated_dataset = pd.concat([concated_dataset, dfDummies_LotConfig], axis=1)
concated_dataset = concated_dataset.drop(['LotConfig'], axis=1)


In [175]:
concated_dataset["LotShape"]=pd.Categorical(concated_dataset["LotShape"])
dfDummies_LotShape = pd.get_dummies(concated_dataset["LotShape"], prefix = 'LotShape')
concated_dataset = pd.concat([concated_dataset, dfDummies_LotShape], axis=1)
concated_dataset = concated_dataset.drop(['LotShape'], axis=1)


In [176]:
concated_dataset["Neighborhood"]=pd.Categorical(concated_dataset["Neighborhood"])
dfDummies_Neighborhood = pd.get_dummies(concated_dataset["Neighborhood"], prefix = 'Neighborhood')
concated_dataset = pd.concat([concated_dataset, dfDummies_Neighborhood], axis=1)
concated_dataset = concated_dataset.drop(['Neighborhood'], axis=1)


In [177]:
concated_dataset["PavedDrive"]=pd.Categorical(concated_dataset["PavedDrive"])
dfDummies_PavedDrive = pd.get_dummies(concated_dataset["PavedDrive"], prefix = 'PavedDrive')
concated_dataset = pd.concat([concated_dataset, dfDummies_PavedDrive], axis=1)
concated_dataset = concated_dataset.drop(['PavedDrive'], axis=1)


In [178]:
concated_dataset["RoofMatl"]=pd.Categorical(concated_dataset["RoofMatl"])
dfDummies_RoofMatl = pd.get_dummies(concated_dataset["RoofMatl"], prefix = 'RoofMatl')
concated_dataset = pd.concat([concated_dataset, dfDummies_RoofMatl], axis=1)
concated_dataset = concated_dataset.drop(['RoofMatl'], axis=1)


In [179]:
concated_dataset["RoofStyle"]=pd.Categorical(concated_dataset["RoofStyle"])
dfDummies_RoofStyle = pd.get_dummies(concated_dataset["RoofStyle"], prefix = 'RoofStyle')
concated_dataset = pd.concat([concated_dataset, dfDummies_RoofStyle], axis=1)
concated_dataset = concated_dataset.drop(['RoofStyle'], axis=1)


In [180]:
concated_dataset["SaleCondition"]=pd.Categorical(concated_dataset["SaleCondition"])
dfDummies_SaleCondition = pd.get_dummies(concated_dataset["SaleCondition"], prefix = 'SaleCondition')
concated_dataset = pd.concat([concated_dataset, dfDummies_SaleCondition], axis=1)
concated_dataset = concated_dataset.drop(['SaleCondition'], axis=1)


In [181]:
concated_dataset["Street"]=pd.Categorical(concated_dataset["Street"])
dfDummies_Street = pd.get_dummies(concated_dataset["Street"], prefix = 'Street')
concated_dataset = pd.concat([concated_dataset, dfDummies_Street], axis=1)
concated_dataset = concated_dataset.drop(['Street'], axis=1)


In [182]:
concated_dataset['YrSold'] = concated_dataset['YrSold'].astype(str)
concated_dataset['MoSold'] = concated_dataset['MoSold'].astype(str)

In [183]:
concated_dataset["YrSold"]=pd.Categorical(concated_dataset["YrSold"])
dfDummies_YrSold = pd.get_dummies(concated_dataset["YrSold"], prefix = 'YrSold')
concated_dataset = pd.concat([concated_dataset, dfDummies_YrSold], axis=1)
concated_dataset = concated_dataset.drop(['YrSold'], axis=1)


In [184]:
concated_dataset["MoSold"]=pd.Categorical(concated_dataset["MoSold"])
dfDummies_MoSold = pd.get_dummies(concated_dataset["MoSold"], prefix = 'MoSold')
concated_dataset = pd.concat([concated_dataset, dfDummies_MoSold], axis=1)
concated_dataset = concated_dataset.drop(['MoSold'], axis=1)


In [185]:
concated_dataset.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9
0,856,854,0,0,3,3,0,706.0,0.0,6,...,0,0,1,0,0,0,0,0,0,0
1,1262,0,0,0,3,3,3,978.0,0.0,5,...,0,0,0,0,0,1,0,0,0,0
2,920,866,0,0,3,3,1,486.0,0.0,6,...,0,0,0,0,0,0,0,0,0,1
3,961,756,0,0,3,4,0,216.0,0.0,5,...,0,0,1,0,0,0,0,0,0,0
4,1145,1053,0,0,4,3,2,655.0,0.0,6,...,0,1,0,0,0,0,0,0,0,0


In [186]:
concated_dataset.select_dtypes(include=['object']).columns


Index([], dtype='object')

In [187]:
concated_dataset['TotalSF'] = concated_dataset['TotalBsmtSF'] + concated_dataset['1stFlrSF'] + concated_dataset['2ndFlrSF']

In [188]:
from scipy.stats import skew
from scipy import stats

In [204]:
numeric_feats = concated_dataset.dtypes[concated_dataset.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = concated_dataset[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skewed Features' :skewed_feats})
skewness.head()

,Skewed Features
MSSubClass_150,54.000003
Exterior1st_ImStucc,54.000003
RoofMatl_Membran,54.000003
Condition2_RRAn,54.000003
MiscFeature_TenC,54.000003


In [205]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    concated_dataset[feat] = boxcox1p(concated_dataset[feat], lam)
    concated_dataset[feat] += 1

There are 266 skewed numerical features to Box Cox transform


In [206]:
concated_dataset.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,TotalSF
0,12.692623,12.686189,1.0,1.0,2.540963,2.540963,1.000000,12.170327,1.0,3.259674,...,1.000000,1.730463,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,15.976591
1,13.792276,1.000000,1.0,1.0,2.540963,2.540963,2.540963,13.062832,1.0,3.055642,...,1.000000,1.000000,1.0,1.0,1.730463,1.0,1.0,1.0,1.000000,15.923100
2,12.892039,12.724598,1.0,1.0,2.540963,2.540963,1.730463,11.200343,1.0,3.259674,...,1.000000,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.730463,16.149678
3,13.013683,12.354094,1.0,1.0,2.540963,2.820334,1.000000,9.274266,1.0,3.055642,...,1.000000,1.730463,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,15.857121
4,13.510588,13.271365,1.0,1.0,2.820334,2.540963,2.194318,11.971129,1.0,3.259674,...,1.730463,1.000000,1.0,1.0,1.000000,1.0,1.0,1.0,1.000000,16.852312


In [207]:
from sklearn.preprocessing import StandardScaler
scaled_features_cn = StandardScaler().fit_transform(concated_dataset.values)
scaled_features_train_postprocess= pd.DataFrame(scaled_features_cn, index=concated_dataset.index, columns=concated_dataset.columns)


In [208]:
train_postprocess = concated_dataset[:train_index]
test_postprocess = concated_dataset[test_index+1:]

In [209]:
x_train = train_postprocess.values.astype(np.float32)


In [210]:
x_train.shape

(1460, 266)

In [211]:
y_train=train_labels.astype(np.float32)

In [212]:
x_test = test_postprocess.values.astype(np.float32)
x_test.shape

(1459, 266)

In [213]:
import tensorflow as tf

In [214]:
m,n=x_train.shape

epochs=500
learning_rate=0.01

from numpy import c_
x_train_bised=np.c_[np.ones((m,1)),x_train]
X_tf_train=tf.constant(x_train_bised,dtype=tf.float32,name="X_tf_train")


y_tf_train=tf.constant(y_train.reshape(-1,1),dtype=tf.float32,name="y_tf_train")
weight=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="weight")


y_pred_train=tf.matmul(X_tf_train,weight,name="y_pred_train")

loss=y_pred_train-y_tf_train
mse=tf.reduce_mean(tf.square(loss),name="mse")
gradient=2/m*tf.matmul(tf.transpose(X_tf_train),loss)



training=tf.assign(weight,weight-learning_rate*gradient)

init=tf.global_variables_initializer()



with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        if epoch % 100 == 0:
            print("Epoch",epoch,"MSe =",mse.eval())
        sess.run(training)
    best_weight=weight.eval()

Epoch 0 MSe = 39034880000.0
Epoch 100 MSe = nan
Epoch 200 MSe = nan
Epoch 300 MSe = nan
Epoch 400 MSe = nan


In [198]:
print(X_tf_train)

Tensor("X_tf_train_1:0", shape=(1460, 267), dtype=float32)


numpy.ndarray

In [202]:
house_data_plus_bias=np.c_[np.ones((m1,1)),housing.data]